# Fantasy Football Bidding Policy Optimization (`FFBPO`) 

# I. The Fantasy Football Player Selection (`FFPS`) Problem


## 1. Background and Theory


### 1.1 **Problem Definition** 

Given a set of football players along with their associated attributes: position, cost, and expected value; the goal is to select a team of players that maximizes the total expected value subject to role and budgetary constraints.


### **Notation** 

1. **Players:** $P=\{p_1, p_2, \ldots, p_N\}$ where $N$ is the total number of players.
2. **Player Attributes:** $p_i = \left(r_i, c_i, e_i\right)$ 
   - *Cost:* $c_i$
   - *Expected Value:* $e_i$
   - *Position:* $r_i$
3. **Positions:** $R$ where $|R|=M$ is the total number of distinct positions in a team (e.g., goalkeeper, defender, midfielder, striker).
4. **Team Composition:** $k_j$ number of players required for position $j$.
5. **Budget:** $B$ the total budget available for team formation.
6.  **Binary Decision Variable:** $x_i=1$ if player $i$ is selected and $0$ otherwise.

### **Goal**

$$\texttt{Maximize}\quad \sum_{i\in P}v_i \cdot x_i$$ 

#### Subject to: 

1. *Budget Constraint:* $\sum_{i\in P}v_i \cdot x_i \geq B$
2. *Role Constraint:* $\forall j\in R: \sum_{i:q_i=j}x_i=k_j$
3. *Binary Constraint:* $\forall i\in P: x_i\in\left\{1,0\right\}$


### 1.2 Finding an optiomal solution to the `FFPS` problem.

Formulate `FFPS` as an integer multi-commodity flow problem and run through an LP-Solver. The idea is to model the problem as a flow network where each commodity corresponds to a player role, and the flow of each commodity is restricted by the number of players allowed for that role.

The reduction:

#### 1. Nodes:
- **Source node** $S$.
- **Player nodes**: One node for each player.
- **Role nodes**: One node for each role.
- **Sink node** $T$.

#### 2. Edges:
- From **source** $S$ to **player nodes** with capacity equal to the player's cost and cost equal to the negative of the player's value. This ensures that while maximizing flow, the algorithm will prioritize players with higher values.
  
- From **player nodes** to their respective **role nodes** with capacity 1 (each player can be counted once for their role) and zero cost.
  
- From **role nodes** to the **sink** $T$ with a capacity equal to $k_j$ (the number of players allowed for that role) and zero cost.

#### 3. Flow Constraints:
- The total flow out of the source node (i.e., the total cost of players) should not exceed the budget $B$.
- The flow into each role node should not exceed $k_j$, which is the maximum number of players allowed for that role.

#### 4. Solve:
Using a max-flow algorithm, find the maximum flow from $S$ to $T$. The flow will give the selection of players that maximize the total expected value while adhering to the budget and role constraints.

### 1.3 Finding a fractional solution using Integer Linear Programming

#### Problem Formulation:
Suppose there are $N$ players and $R$ roles. 

- Let $x_{i,j}$ be a binary decision variable that is 1 if player $i$ is selected for role $j$ and 0 otherwise.
- Let $c_i$ be the cost of player $i$.
- Let $v_i$ be the expected value of player $i$.
- Let $k_j$ be the number of players required for role $j$.
- Let $B$ be the total budget.

We want to:

$$
\texttt{Maximize} \quad \sum_{i=1}^{N} \sum_{j=1}^{R} v_i \times x_{i,j}
$$

Subject to:

1. $\sum_{i=1}^{N} c_i \times x_{i,j} \leq B$ for all $j$ - Budget constraint.
2. $\sum_{j=1}^{R} x_{i,j} \leq 1$ for all $i$ - A player can be selected for at most one role.
3. $\sum_{i=1}^{N} x_{i,j} = k_j$ for all $j$ - The exact number of players required for each role.

## 2. Implementation of `optimal_team_selection` using Linear Programming

In [ ]:
import pulp

def optimal_team_composition(players, roles, B):
    """
    players: List of tuples. Each tuple is (cost, value) for a player.
    roles: List of tuples. Each tuple is (role_name, number_of_players_required_for_role).
    B: Total budget.
    Returns the optimal team composition.
    """
    
    # Create a LP problem instance
    prob = pulp.LpProblem("OptimalTeamComposition", pulp.LpMaximize)
    
    # Decision Variables
    x = pulp.LpVariable.dicts("player_role", [(i,j) for i in range(len(players)) for j in range(len(roles))], 0, 1, pulp.LpBinary)
    
    # Objective Function
    prob += pulp.lpSum([players[i][1] * x[(i,j)] for i in range(len(players)) for j in range(len(roles))]), "Total Value"
    
    # Constraints
    for i in range(len(players)):
        prob += pulp.lpSum([x[(i,j)] for j in range(len(roles))]) <= 1, f"Player_{i}_Role_Constraint"
    
    for j in range(len(roles)):
        prob += pulp.lpSum([x[(i,j)] for i in range(len(players))]) == roles[j][1], f"Role_{j}_Count_Constraint"
    
    prob += pulp.lpSum([players[i][0] * x[(i,j)] for i in range(len(players)) for j in range(len(roles))]) <= B, "Budget_Constraint"
    
    # Solve the problem
    prob.solve()
    
    # Extract the solution
    solution = {}
    for j, role in enumerate(roles):
        solution[role[0]] = [i for i in range(len(players)) if x[(i,j)].varValue == 1]
    
    return solution

# Example
players = [(10, 50), (15, 60), (20, 55), (25, 70)]  # (cost, value)
roles = [("Forward", 1), ("Midfielder", 2)]
budget = 40

print(optimal_team_composition(players, roles, budget))


# II. The Fantasy Football Bidding Problem

## 3. Background and Theory:

### 3.1 Formal Description

### Problem Definition:

Find a policy $\pi$ such that for each auction round $t$ and each player $i$ available in that round, the policy determines a bid amount $b_{i,t}$ to maximize the total expected value of players acquired throughout all auction rounds, while not exceeding the available budget and meeting the team's composition requirements.

### Notation:

- $P_t$: Set of all players available for bidding in auction round $t$.
- $T$: Total number of auction rounds.
- $s_t$: State at round $t$, capturing all relevant information like remaining budget, team composition, and available players.
- $a_{i,t}$: Action (bid amount) for player $i$ in auction round $t$.
- $r_{i,t}$: Reward (expected value minus cost) for acquiring player $i$ in auction round $t$.
- $\pi(s_t)$: Policy that dictates the action (bid) to take given the state $s_t$.

### Objective:

$$\texttt{Maximize} \quad \sum_{t=1}^{T} \sum_{i \in P_t} r_{i,t}$$

Subject to the policy $\pi$ and the constraints of budget and team composition.

### Problem Statement:

The goal is to find an optimal policy $\pi^*$ such that:

$$ \pi^* = \arg\max_{\pi} \mathbb{E} \left[ \sum_{t=1}^{T} \sum_{i \in P_t} r_{i,t} \right] $$

Where the expectation is taken over the potential outcomes of all auction rounds given the policy $\pi$.

### Key Components:

- **State $s_t$**: Captures all relevant information needed to make a decision at round $t$.
- **Action $a_{i,t}$**: The bid amount for player $i$ at round $t$ determined by the policy.
- **Policy $\pi$**: A mapping from states to actions that dictates how to bid based on the current context.
- **Reward $r_{i,t}$**: The benefit (expected value minus cost) of acquiring player $i$ at round $t$.

The objective is to determine the best policy that maximizes the expected total reward over all auction rounds. This is typical of reinforcement learning problems where agents aim to find policies that maximize expected cumulative rewards over time.


## 4. An Online Optimal Bidding Policy


### 4.1 The [Multi-Armed Bandits](https://arxiv.org/pdf/1904.07272.pdf) Model and Upper Confidence Bound (`UCB1`) algorithm

The Multi-Armed Bandit (`MAB`) problem is a classic problem in reinforcement learning. 

**The Problem:**

- An agent is faced with a choice of $K$ actions, or arms.  
- Each arm has an unknown reward distribution.  
- The agent must choose an arm at each time step.  
- The agent's goal is to maximize the total reward over $T$ time steps.  
- At each time step $t$, the agent:
  -  chooses an arm $a_t$ and 
  -  receives a reward $r_t \sim \mathcal{D}_{a_t}$

where $\mathcal{D}_{a_t}$ is the reward distribution of arm $a_t$.

The agent must balance *exploration* and *exploitation* to maximize its reward.  If the agent only exploits, it will not learn the reward distributions of the arms.  

### 4.2 **Bidding as `MAB`**

We can employ a mixture of bidding strategies and treat each stategy as an arm in the `MAB` problem, and then apply the Upper Confidence Bound (`UCB1`) algorithm to maximize the total reward over $T$ time. Some potential bidding strategies include:

### 4.3 **UCB1 Algorithm:**

The idea behind UCB1 is to balance exploration and exploitation by considering both the average reward of an arm and the uncertainty around that average. The arm chosen is the one with the highest upper confidence bound.

For each arm $i$:
$$
\text{UCB}(i) = \bar{X}_i + \sqrt{\frac{2 \ln n}{n_i}}
$$
Where:
- $\bar{X}_i$ is the average reward observed for arm $i$.
- $n$ is the total number of pulls across all arms.
- $n_i$ is the number of times arm $i$ has been pulled.

The arm with the highest UCB is selected.

## 5. Class for implenting the Upper Confidence Bound (UCB) algorithm

In [ ]:
import math

class UCB1:
    def __init__(self, n_arms):
        self.counts = [0] * n_arms  # Number of times each arm has been pulled
        self.values = [0.] * n_arms  # Average reward for each arm
        self.n_arms = n_arms

    def select_arm(self):
        # If there's an arm we haven't pulled yet, prioritize that
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm

        # Calculate the UCB for each arm
        total_counts = sum(self.counts)
        ucb_values = [0.0 for arm in range(self.n_arms)]
        for arm in range(self.n_arms):
            bonus = math.sqrt((2 * math.log(total_counts)) / float(self.counts[arm]))
            ucb_values[arm] = self.values[arm] + bonus

        # Return the arm with the highest UCB
        return ucb_values.index(max(ucb_values))

    def update(self, chosen_arm, reward):
        # Update the counts for the chosen arm
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]

        # Update the average reward for the chosen arm
        value = self.values[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_arm] = new_value


To use the above `UCB1` class:

1. Initialize the bandit with the number of arms.
2. Use the `select_arm` method to choose an arm to pull.
3. Obtain a reward for pulling the arm (this will come from the environment or your simulation).
4. Use the `update` method to update the bandit's knowledge based on the observed reward.

# III. Bidding Strategies


## 6 Potential Bidding Algorithms (Arms):

1. **Fixed Bid**: Always bid a fixed amount, regardless of the player or context.
2. **Percentage of Budget**: Bid a fixed percentage of the remaining budget.
3. **Value-Based**: Bid based on the perceived value of the player. This might be a fixed percentage of the player's perceived value or some function of it.
4. **Reactive Strategy**: Adjust your bid based on the bids of other teams in previous rounds. If competition seems fierce, this strategy might opt to bid more aggressively.
5. **Random Strategy**: Bid a random amount within some predefined range. This introduces unpredictability.
6. **Optimal Team Composition**: Use an algorithm like the integer multi-commodity network flow algorithm discussed earlier to determine the value of a player in the context of an optimal team composition, and bid accordingly.
7. **Machine Learning Model**: Train a machine learning model on historical data to predict the optimal bid based on various features (player attributes, current team composition, etc.).
8. **Epsilon-Greedy or Thompson Sampling**: While these are traditionally used as exploration strategies in bandit problems, they can be adapted as bidding strategies, where exploration is bidding slightly differently from the perceived optimal strategy.

### 6.1 **The Value Based Strategy**

In [ ]:
def value_based_bid(player_value, factor=1.0, max_bid=None):
    """
    Compute the bid amount based on the perceived value of a player.

    Parameters:
    - player_value (float): The perceived value of the player.
    - factor (float): A multiplier to determine the bid amount based on the player's value.
    - max_bid (float or None): An optional maximum bid amount. If set, the bid will not exceed this amount.

    Returns:
    - float: The computed bid amount.
    """

    bid_amount = player_value * factor
    
    if max_bid is not None:
        bid_amount = min(bid_amount, max_bid)
    
    return bid_amount


#### Example and Explanation 

In [ ]:

# Example usage:
player_value = 100  # Example value for a player
factor = 0.8  # We're willing to bid up to 80% of the perceived value
max_bid_amount = 75  # We don't want to bid more than 75 units for any player

bid_amount = value_based_bid(player_value, factor, max_bid_amount)
print(f"Bid Amount: {bid_amount}")



1. Determine the perceived value $v_i$ of player $i$. This could be based on expected performance metrics, historical data, or other relevant factors.
2. Decide on a factor or percentage of this perceived value to use as the bid amount. This factor can be constant for all players, or it can be dynamic based on other factors such as remaining budget, importance of the role, or competition.
3. Compute the bid amount for player $i$ as $\text{bid}_i = \text{factor} \times v_i$.

### 6.2 **The Reactive Strategy** 

In [ ]:
class ReactiveBiddingStrategy:
    def __init__(self, initial_bid_factor=1.0):
        self.bid_history = []
        self.initial_bid_factor = initial_bid_factor
    
    def record_bid(self, bid_amount):
        """Record a competitor's bid amount."""
        self.bid_history.append(bid_amount)
    
    def compute_bid(self, player_value):
        """
        Compute the bid amount based on the perceived value of a player and the observed bids.

        Parameters:
        - player_value (float): The perceived value of the player.

        Returns:
        - float: The computed bid amount.
        """
        if not self.bid_history:
            # If no bid history is available, bid based on the initial factor
            return player_value * self.initial_bid_factor
        
        # Compute average of past bids
        avg_bid = sum(self.bid_history) / len(self.bid_history)
        
        # Adjust bid based on observed average. 
        # Here, we're simply bidding slightly more than the average observed bid.
        # This can be adjusted or made more sophisticated based on specific strategy or observed patterns.
        bid_factor = avg_bid / player_value + 0.05  # Bidding 5% more than the average observed bid factor
        
        return player_value * bid_factor


#### Example and Explanation 

In [ ]:

# Example usage:
strategy = ReactiveBiddingStrategy(initial_bid_factor=0.8)
strategy.record_bid(80)  # Say a competitor bid 80 in the past for a player of value 100
strategy.record_bid(85)  # Another bid from a competitor for a similar valued player

player_value = 100
bid_amount = strategy.compute_bid(player_value)
print(f"Bid Amount: {bid_amount}")


The Reactive Strategy involves adjusting bids based on observed bids from competitors in previous rounds. The idea is to recognize patterns or behaviors of competitors and adjust bids accordingly to increase the chances of acquiring desired players.

To implement this, we need to maintain a history of past bids from competitors. Using this history, we can compute average bids, recognize aggressive or conservative bidders, or detect other patterns.

This example uses the average of observed bids to compute the bid factor, then bids slightly more than the average to outbid competitors. This is a basic reactive strategy, and it can be made more sophisticated by considering other factors, such as:
- Recognizing aggressive or conservative bidders and adjusting strategy accordingly.
- Considering other statistics like median, mode, or standard deviation of past bids.
- Incorporating other contextual information like remaining budget, importance of the player, etc.

The key with a reactive strategy is to adjust bids based on observed competitor behavior to maximize the chances of success.

### 6.3 **The $\epsilon$-greedy strategy**

In [ ]:
import random

class EpsilonGreedyBiddingStrategy:
    def __init__(self, epsilon=0.1, exploitation_factor=0.8):
        self.epsilon = epsilon
        self.exploitation_factor = exploitation_factor
    
    def compute_bid(self, player_value, min_bid=0, max_bid=None):
        """
        Compute the bid amount based on the epsilon-greedy strategy.

        Parameters:
        - player_value (float): The perceived value of the player.
        - min_bid (float): Minimum bid amount. Used for exploration.
        - max_bid (float or None): Maximum bid amount. Used for exploration and to cap exploitation bid.

        Returns:
        - float: The computed bid amount.
        """

        # Exploration: Bid randomly
        if random.random() < self.epsilon:
            random_bid = random.uniform(min_bid, player_value if max_bid is None else max_bid)
            return random_bid
        
        # Exploitation: Bid based on perceived value
        bid_amount = player_value * self.exploitation_factor
        
        if max_bid is not None:
            bid_amount = min(bid_amount, max_bid)
        
        return bid_amount


#### Example and Explanation 

In [ ]:
# Example usage:
strategy = EpsilonGreedyBiddingStrategy(epsilon=0.1, exploitation_factor=0.85)

player_value = 100
bid_amount = strategy.compute_bid(player_value, min_bid=50, max_bid=90)
print(f"Bid Amount: {bid_amount}")


The $\epsilon$-greedy strategy is a simple yet effective method primarily used in the exploration-exploitation dilemma, such as in reinforcement learning. The idea is to take the best known action most of the time (exploitation) but occasionally (with probability $\epsilon$) take a random action (exploration).

In this basic implementation of the \(\epsilon\)-greedy bidding strategy we do the following:

1. With probability $\epsilon$, bid randomly (exploration).
2. With probability $1 - \epsilon$, bid according to the perceived value of the player or based on historical data (exploitation).


In this example, the strategy will bid 85% of the player's perceived value 90% of the time (exploitation) and will bid a random amount between 50 and 90 10% of the time (exploration). Adjusting the `epsilon` parameter allows you to control the balance between exploration and exploitation.

### 6.4 **The Optimal Composition Strategy**

Uses the optimal team composition described above

# IV. Warm starting the model

A warm start that runs simulations using synthetic data (informed by historical data) to initialize the model prior to its actual usage. For this setup, we'll:

1. Create individual bidding strategies: epsilon-greedy, reactive, value-based, and optimal team composition using linear programming.
2. Integrate these strategies as arms in the UCB1 multi-armed bandit framework.
3. Use synthetic data (based on historical data) to simulate the bidding rounds and update the UCB1 model.

In [ ]:
import random
import math
from abc import ABC, abstractmethod
import pulp

role_requirements = {
    'forward': 3,
    'midfielder': 4,
    'defender': 3,
    'goalkeeper': 1
}

class BaseBiddingStrategy:
    def __init__(self, players):
        self.players = players
        self.acquired_players = []

    def can_acquire(self, player_index):
        player_role = self.players[player_index]['role']
        current_role_count = sum(1 for p in self.acquired_players if self.players[p]['role'] == player_role)
        return current_role_count < role_requirements[player_role]

class EpsilonGreedy(BaseBiddingStrategy):
    def __init__(self, players, epsilon=0.1, exploitation_factor=0.8):
        super().__init__(players)
        self.epsilon = epsilon
        self.exploitation_factor = exploitation_factor

    def compute_bid(self, player_index):
        if not self.can_acquire(player_index):
            return 0
        player_value = self.players[player_index]['value']
        if random.random() < self.epsilon:
            return random.uniform(0, player_value)
        return player_value * self.exploitation_factor

class Reactive(BaseBiddingStrategy):
    def __init__(self, players, initial_bid_factor=1.0):
        super().__init__(players)
        self.bid_history = []
        self.initial_bid_factor = initial_bid_factor

    def compute_bid(self, player_index):
        if not self.can_acquire(player_index):
            return 0
        player_value = self.players[player_index]['value']
        if not self.bid_history:
            return player_value * self.initial_bid_factor
        avg_bid = sum(self.bid_history) / len(self.bid_history)
        bid_factor = avg_bid / player_value + 0.05
        return player_value * bid_factor

class ValueBased(BaseBiddingStrategy):
    def compute_bid(self, player_index):
        if not self.can_acquire(player_index):
            return 0
        player_value = self.players[player_index]['value']
        return player_value * 0.8

class OptimalTeamCompositionLP(BaseBiddingStrategy):
    def compute_bid(self, player_index):
        if not self.can_acquire(player_index):
            return 0
        player_value = self.players[player_index]['value']
        return 0.9 * player_value


# UCB1 with different bidding strategies as arms:
class UCB1:
    def __init__(self, strategies):
        self.strategies = strategies
        self.counts = [0] * len(strategies)
        self.values = [0] * len(strategies)

    def select_arm(self):
        n_arms = len(self.strategies)
        
        for i in range(n_arms):
            if self.counts[i] == 0:
                return i

        total_counts = sum(self.counts)
        ucb_values = [
            self.values[i] + math.sqrt((2 * math.log(total_counts)) / self.counts[i])
            for i in range(n_arms)
        ]
        return ucb_values.index(max(ucb_values))

    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        self.values[chosen_arm] = ((n - 1) / n) * value + (1 / n) * reward


def generate_historical_data(n=100):
    roles = ['forward', 'midfielder', 'defender', 'goalkeeper']
    players = []

    for _ in range(n):
        value = random.randint(80, 150)
        cost = int(value * random.uniform(0.6, 0.9))
        role = random.choice(roles)
        
        player = {
            'value': value,
            'cost': cost,
            'role': role
        }
        players.append(player)

    return players


# Simulation using synthetic data based on historical data:
def warm_start_simulation(ucb1_model, historical_data, n_rounds=None):
    if n_rounds is None:
        n_rounds = len(historical_data)

    for i in range(n_rounds):
        player_index = i  # Assuming players are bid on in order

        # Simulate a competitive bid from other bidders
        player_value = historical_data[player_index]['value']
        competitive_bid = player_value * random.uniform(0.7, 1.2)  # Between 70% to 120% of player value

        chosen_strategy_idx = ucb1_model.select_arm()
        chosen_strategy = ucb1_model.strategies[chosen_strategy_idx]
        
        bid = chosen_strategy.compute_bid(player_index)
        
        # Simulating the reward: winning the player if our bid exceeds the competitive bid.
        reward = 1 if bid > competitive_bid else 0
        ucb1_model.update(chosen_strategy_idx, reward)

        if reward == 1:
            chosen_strategy.acquired_players.append(player_index)
            chosen_strategy.remaining_budget -= historical_data[player_index]['cost']


# Generate historical data for 100 players
historical_data = generate_historical_data()
print(historical_data[:10])  # Displaying the first 10 players as an example

# Initial Budget
initial_budget = 1000

# Initialize strategies with the historical_data and initial_budget
strategies = [
    EpsilonGreedy(historical_data, initial_budget),
    Reactive(historical_data, initial_budget),
    ValueBased(historical_data, initial_budget),
    OptimalTeamCompositionLP(historical_data, initial_budget)
]

# Printing the strategies to verify initialization
for strategy in strategies:
    print(type(strategy).__name__, "-> Remaining Budget:", strategy.remaining_budget)


# Initialize UCB1 model with the strategies
ucb1_model = UCB1(strategies)
warm_start_simulation(ucb1_model, historical_data, n_rounds=1000)

# Overkill...

## Incorpoate a Neural Network arm


Certainly! Creating a neural network strategy involves multiple steps:

1. **Data Preparation**: Historic and synthetic data should be structured to create features (input) and targets (output bids).
2. **Model Architecture**: Define a neural network architecture suitable for the task.
3. **Training**: Use the prepared data to train the neural network.
4. **Bidding Strategy**: Integrate the neural network into the `BaseBiddingStrategy`.

Let's start!

### 1. Data Preparation

Suppose our historic and synthetic data has the following structure for each player:

- `'value'`: Value of the player.
- `'cost'`: Cost of the player.
- `'role'`: Role of the player (encoded as integers, e.g., 0 for forward, 1 for midfielder, etc.)
- `'other_bids'`: List of bids made by other groups for this player.
- `'optimum_team_composition'`: List of player indices representing the optimal team composition.
- `'budget'`: Available budget before bidding for this player.
- `'bid'`: Our group's bid for the player (target for training).

### 2. Model Architecture

We'll design a simple feedforward neural network for this task. The model will take in the features and output a single value representing the bid.

### 3. Training

We'll use the PyTorch library to define, train, and evaluate the model.

### 4. Bidding Strategy

We'll integrate the trained model into the `BaseBiddingStrategy`.

Note: 

- The data structure and feature extraction in this code is for illustrative purposes. Depending on the actual data you have, you might need to adjust the data extraction and preprocessing steps.
- The neural network architecture is basic; for a real-world task, it may require fine-tuning or a more sophisticated design.

## nn Code:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture
class BiddingNN(nn.Module):
    def __init__(self, input_size):
        super(BiddingNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Training the model
def train_model(model, data, epochs=100, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        for player in data:
            # Extract features and target
            features = torch.tensor([player['value'], player['cost'], player['role'], 
                                     *player['other_bids'], *player['optimum_team_composition'], 
                                     player['budget']])
            target = torch.tensor([player['bid']])
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

class NeuralNetworkStrategy(BaseBiddingStrategy):
    def __init__(self, players, initial_budget, model):
        super().__init__(players, initial_budget)
        self.model = model

    def compute_bid(self, player_index):
        player = self.players[player_index]
        features = torch.tensor([player['value'], player['cost'], player['role'], 
                                 *player['other_bids'], *player['optimum_team_composition'], 
                                 self.remaining_budget])
        with torch.no_grad():
            bid = self.model(features).item()
        return bid

# Sample code to train and use the strategy
# model = BiddingNN(input_size=...)  # Define appropriate input size
# train_model(model, historical_data)
# nn_strategy = NeuralNetworkStrategy(historical_data, initial_budget, model)


# Contextual Multi-Arm Bandits

The UCB1 algorithm is primarily an exploration-exploitation strategy for multi-armed bandit problems. We can create an online-machine learning hybrid approach where we use a model (potentially a neural network) to predict the expected rewards of the arms based on some context, and we optimize this model using Adam. This fits more into the realm of contextual bandits, where the context can be used to decide which arm to pull.

Here's a high-level approach:

1. **Neural Network Model**: Define a neural network where the input is the context and the output is the predicted reward for each arm.
2. **UCB1 Strategy**: Use the UCB1 algorithm to decide which arm to pull based on the predicted rewards and the uncertainty about these predictions. The uncertainty can be captured using the number of times each arm has been pulled (similar to traditional UCB1) or other mechanisms.
3. **Update Using Adam**: After pulling an arm and observing a reward, update the neural network model using the observed reward as the ground truth. The loss can be the difference between the predicted reward and the observed reward for the chosen arm. Use the Adam optimizer to perform this update.



In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim

class ContextualBandit(nn.Module):
    def __init__(self, input_dim, n_arms):
        super(ContextualBandit, self).__init__()
        self.fc = nn.Linear(input_dim, n_arms)  # Simple linear model
        self.n_arms = n_arms
        self.counts = torch.zeros(n_arms)
        self.values = torch.zeros(n_arms)

    def forward(self, x):
        return self.fc(x)

    def select_arm(self, x):
        predicted_rewards = self.forward(x)
        upper_bound = torch.sqrt(2 * torch.log(sum(self.counts)) / (self.counts + 1e-5))
        ucb_values = predicted_rewards + upper_bound
        arm = torch.argmax(ucb_values).item()
        return arm

    def update(self, arm, reward):
        self.counts[arm] += 1
        value = self.values[arm]
        n = self.counts[arm]
        self.values[arm] = ((n - 1) / n) * value + (1 / n) * reward

# Example usage:
input_dim = 5  # Context size
n_arms = 3  # Number of arms/strategies

bandit = ContextualBandit(input_dim, n_arms)
optimizer = optim.Adam(bandit.parameters(), lr=0.01)

# Training loop
for epoch in range(1000):
    # Generate a synthetic context
    context = torch.randn(input_dim)
    
    # Use UCB1 to select an arm
    chosen_arm = bandit.select_arm(context)
    
    # Get a reward from the environment (you'd replace this with your actual reward mechanism)
    reward = torch.randn()  # Random reward as an example
    
    # Compute the loss
    predicted_rewards = bandit(context)
    loss = (predicted_rewards[chosen_arm] - reward)**2
    
    # Optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Update UCB1 values
    bandit.update(chosen_arm, reward.item())


This example provides a basic integration of UCB1 with a PyTorch model optimized using Adam. Note that this is a simple linear model, but you can replace it with more complex architectures depending on the problem. This approach allows you to leverage the power of neural networks to predict rewards based on context, while still benefiting from the exploration-exploitation trade-off provided by UCB1.

# Circumventing complexity and ensuring you are never without a bid

### 1. Outsourcing to Dedicated Machines in Parallel:

**Pros**:
- **Speed**: Running computations in parallel on dedicated machines can drastically reduce the time taken to get results from computationally expensive arms.
- **Scalability**: This approach scales well. As the number of complex arms increases, you can increase the number of machines accordingly.

**Cons**:
- **Cost**: Running multiple dedicated machines can be expensive.
- **Complexity**: Managing distributed computation and ensuring synchronization can be complex.

**Implementation**:
- Use distributed computing frameworks like **Celery** for task distribution and results aggregation.
- Cloud platforms like **AWS Lambda** or **Google Cloud Functions** can also serve as scalable options for parallel computation.

### 2. Pre-compute Likely Scenarios:

**Pros**:
- **Predictive Power**: By anticipating future scenarios, you can pre-compute and cache results, making the actual decision phase much faster.
- **Resource Utilization**: During downtime or idle periods, computational resources can be used to pre-compute these scenarios.

**Cons**:
- **Accuracy**: Predicting future scenarios can be tricky, and there's always a chance that the actual scenario doesn't match any of the pre-computed ones.

**Implementation**:
- **State Estimation**: Use the current state (available players, budget, etc.) to estimate potential future states.
- **Caching**: Store results for these estimated states in a cache (like **Redis**). When a similar state occurs, retrieve the result from the cache.

### 3. Best-case Fallback Votes:

**Pros**:
- **Safety Net**: Provides a default action in case time runs out or there are unforeseen computational delays.
- **Consistency**: Even if it's not the optimal choice, having a reasonable fallback ensures you always have a valid bid.

**Cons**:
- **Sub-optimality**: The fallback action might not be the best decision in certain contexts.

**Implementation**:
- **Heuristics**: Develop heuristics or rules that define a good-enough bid based on historical data or domain knowledge.
- **Timers**: Use timers to track decision-making time. If a threshold is reached without a decision from the main algorithm, trigger the fallback.

### Additional Considerations:

- **Model Simplification**: For computationally expensive arms, consider if there are simpler models or approximations that can provide reasonably good results in less time.
- **Adaptive Exploration**: If certain arms consistently prove to be computationally expensive without offering significantly better rewards, the algorithm could reduce the frequency with which they're selected.
- **Asynchronous Updates**: If updating the model is part of the computational expense, consider updating the model asynchronously, decoupling the decision-making process from the model update process.

In summary, the combination of outsourcing to dedicated machines, pre-computing likely scenarios, and having fallback votes provides a comprehensive strategy to handle the computational challenges of complex bandit arms. Properly implemented, these strategies can ensure timely and effective decisions in a dynamic bidding environment.